In [1]:
### Distance Matrix to MDS

In [2]:
import pandas as pd
import numpy as np

In [3]:
### Import distance matrix and remove Row labels
df = pd.read_table("Data/ssRNA_complete_genomes_100_subsample_50000.dist")
df = df.drop("#query", axis=1)
### Get filenames from headers
INDEX = list(df)
print(len(INDEX))

3900


In [4]:
import pandas as pd
df_Accessions_100 = pd.read_csv("Data/df_ssRNA_complete_100_subsample.tsv", sep="\t")
len(df_Accessions_100)

3900

In [5]:
Acc_to_species = dict(zip(df_Accessions_100["accs"], df_Accessions_100["species"]))

In [6]:
import pickle
with open('Data/Accessions_100.pickle', 'rb') as handle:
    Accessions = pickle.load(handle)
len(Accessions)

3900

In [7]:
Sub_positions = []
Sub_accessions = []
for Acc in INDEX:
    if Acc in Accessions:
        Sub_positions.append(int(INDEX.index(Acc)))
        Sub_accessions.append(Acc)
print(len(Sub_positions))

3900


In [8]:
Sub_species = []
for Acc in Sub_accessions:
    Sub_species.append(Acc_to_species[Acc])

In [9]:
Submatrix = df.iloc[Sub_positions, Sub_positions]

In [10]:
def seriation(Z,N,cur_index):
    '''
        input:
            - Z is a hierarchical tree (dendrogram)
            - N is the number of points given to the clustering process
            - cur_index is the position in the tree for the recursive traversal
        output:
            - order implied by the hierarchical tree Z
            
        seriation computes the order implied by a hierarchical tree (dendrogram)
    '''
    if cur_index < N:
        return [cur_index]
    else:
        left = int(Z[cur_index-N,0])
        right = int(Z[cur_index-N,1])
        return (seriation(Z,N,left) + seriation(Z,N,right))
    
def compute_serial_matrix(dist_mat,method="ward"):
    '''
        input:
            - dist_mat is a distance matrix
            - method = ["ward","single","average","complete"]
        output:
            - seriated_dist is the input dist_mat,
              but with re-ordered rows and columns
              according to the seriation, i.e. the
              order implied by the hierarchical tree
            - res_order is the order implied by
              the hierarhical tree
            - res_linkage is the hierarhical tree (dendrogram)
        
        compute_serial_matrix transforms a distance matrix into 
        a sorted distance matrix according to the order implied 
        by the hierarchical tree (dendrogram)
    '''
    N = len(dist_mat)
    flat_dist_mat = squareform(dist_mat)
    res_linkage = linkage(flat_dist_mat, method=method)
    res_order = seriation(res_linkage, N, N + N-2)
    seriated_dist = np.zeros((N,N))
    a,b = np.triu_indices(N,k=1)
    seriated_dist[a,b] = dist_mat[ [res_order[i] for i in a], [res_order[j] for j in b]]
    seriated_dist[b,a] = seriated_dist[a,b]
    
    return seriated_dist, res_order, res_linkage

In [11]:
# from sklearn import datasets

from scipy.spatial.distance import pdist, squareform
from scipy.cluster.hierarchy import linkage
from scipy.cluster.hierarchy import dendrogram
from scipy.cluster import hierarchy

import matplotlib as mpl
from matplotlib import pyplot as plt
from matplotlib.pyplot import cm


In [12]:
ordered_dist_mat, res_order, res_linkage = compute_serial_matrix(Submatrix.values,'average')
ordered_dist_mat = ordered_dist_mat[::-1]

In [13]:
### Make Plot ###
fig=plt.figure(facecolor='white')
fig.set_size_inches(30, 20)
fig.set_dpi(300)

### Plot Dendrogram ###
axd = fig.add_axes([0.05, 0.1, 0.35, 0.85])
hierarchy.set_link_color_palette(['#42bb72ff', "#0c2a50ff", "#0c2a50ff", "#0c2a50ff"])


row_dendr = dendrogram(res_linkage, 
                       labels=Sub_accessions, 
                       orientation='left', 
                       color_threshold=0.5, 
                       above_threshold_color='#440154ff')

axd.set_xlabel('Jaccard Distance', fontsize=20, labelpad=20)
axd.tick_params(axis='both', which='major', labelsize=20)
axd.axes.get_yaxis().set_visible(False)


### Plot Heatmap ###
axm = fig.add_axes([0.15, 0.1, 0.9, 0.85])

cax = axm.matshow(ordered_dist_mat, interpolation='nearest', cmap='viridis_r')
axm.grid(False)
axm.tick_params(axis='both', which='major', labelsize=20)
axm.axes.get_yaxis().set_visible(False)
axm.xaxis.tick_bottom()
axm.set_xlabel('Genome number', fontsize=20, labelpad=20)

### Color legend ###
colorbar = fig.colorbar(cax,fraction=0.025)
colorbar.ax.invert_yaxis()
colorbar.set_label(label='Jaccard Distance',size=20, labelpad=20)
colorbar.ax.tick_params(labelsize=20) 

### Figure size and resolution ###
fig.savefig('Visualizations/ssRNA_complete_100_subsample_mash_50000_complete.pdf')